# Forecasting

In [13]:
from feature_generation import *

import pandas as pd

In [14]:
df = pd.read_csv("DATA_sample_atm.csv")
means = df.groupby("HistoryDate").mean()[['CashIn','CashOut']]
means = means[:-135]
means = clean_data(means)

In [15]:
feature_set = get_feature_sets(means, ['CashIn', 'CashOut'])

## Calculating Error

Common way of calculating error in timeseries forecasting is [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)

In [22]:
def mape_error(y_actual, y_pred):
    return 100 * ((y_actual - y_pred).abs() / y_actual).mean()

## Base Error

To evaluate our models and see if they are useful, we need a base error to compare them with. One way of doing this is to use shifted data as predictions.

In [34]:
base_errors = pd.DataFrame(columns=['CashIn_Error', 'CashOut_Error'])
for target in ['CashIn', 'CashOut']:
    windows = get_windows(means[target], 21, drop_t=False)
    for column in windows.columns[1:]:
        base_errors.loc[column, target + '_Error'] = mape_error(windows['t'], windows[column])

In [37]:
base_errors.sort_values(by='CashIn_Error')[:3]

,CashIn_Error,CashOut_Error
t-14,14.631056,35.001829
t-21,14.640567,33.055544
t-7,16.436717,30.902763


In [38]:
base_errors.sort_values(by='CashOut_Error')[:3]

,CashIn_Error,CashOut_Error
t-1,30.247342,29.27443
t-7,16.436717,30.902763
t-21,14.640567,33.055544


### Base errors:
* CashIn: 14.63
* CashOut: 29.27

## Forecasting

### Models to try:
* Random forest
* Lightgbm
* Catboost